# Load data
This code loads the data for 1 subject with the help of methods defined in data_handler.ipynb.
Also prints distribution of the split just to make sure it's stratified.

In [1]:
import ipynb
import torch
from ipynb.fs.full.data_handler import load_subject, extract_labels, extract_action_interval, split_data, split_info, to_device
from ipynb.fs.full.train_model import train_model, accuracy_check

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

data, description = load_subject(1)
labels = extract_labels(description)
data_interval = extract_action_interval(data)

train_data, val_data, test_data, train_labels, val_labels, test_labels = split_data(data_interval, labels)

train_data, val_data, test_data, train_labels, val_labels, test_labels = to_device(train_data, val_data, test_data, train_labels, val_labels, test_labels, device)
split_info(train_data, val_data, test_data, train_labels, val_labels, test_labels)

[99, 99, 99, 99] 
 [13, 13, 14, 14] 
 [13, 13, 12, 12]
25.0 %  25.0 %  25.0 %  25.0 % 
24.074074074074073 %  24.074074074074073 %  25.925925925925924 %  25.925925925925924 % 
26.0 %  26.0 %  24.0 %  24.0 % 


# EEGNet
Implementation of EEGNetfrom (Lawhern et. al.) for the use on "Thinking out loud" dataset.

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim


class EEGNet(nn.Module):
    def __init__(self, interval = "full", dropout = 0.5):
        super(EEGNet, self).__init__()
        self.f1 = 16
        self.d = 4
        self.f2 = self.d*self.f1

        # Block 1
        self.conv1 = nn.Conv2d(in_channels=1, out_channels = self.f1, kernel_size = (1, 128), padding = 0, bias = False) # 128 = half sampling rate, reduces to 2Hz, 16 is probably F1, the number of filters (output channels)
        self.batchnorm1 = nn.BatchNorm2d(self.f1, False)

        self.conv2 = nn.Conv2d(in_channels = self.f1, out_channels = self.f1*self.d, kernel_size = (128, 1), padding = 0, bias = False) # row, column] 128 = number of sensors for the depth wise convolution
        self.batchnorm2 = nn.BatchNorm2d(self.f1*self.d, False) # Not sure this is the right number of features.
        # Elu
        self.elu = nn.ELU()
        # Dropout
        self.dropout = nn.Dropout(dropout)

        self.avgPool1 = nn.AvgPool2d((1,4)) # Reduces sampling rate to 64hz

        # Block 2

        # Separable convolution
        self.depthwise = nn.Conv2d(in_channels = self.f1*self.d, out_channels = self.f1*self.d, kernel_size = (1, 32),
                        bias=False, groups = self.f1*self.d, padding = (0, 16//2)) # Captures 500ms of data at sampling rate 64Hz (32 Hz is 500ms of that)
        self.pointwise = nn.Conv2d(in_channels = self.f1*self.d, out_channels = self.f2, kernel_size = 1) # This results in f2 channels of the output here.
        
        self.batchnorm3 = nn.BatchNorm2d(self.f2, False)

        self.avgPool2 = nn.AvgPool2d((1,8))
        
        # Fully connected classifier
        if interval == "action":
            self.classifier = nn.Linear(896,4, bias = False)
        elif interval == "full":
            self.classifier = nn.Linear(1920,4, bias = False)
        self.softmax = nn.LogSoftmax(dim = 1)


    def forward(self, x): # Dropout 0.5 dropout for within-subject and 0.25 for cross-subject
        # Block 1
        res = self.conv1(x)
        res = self.batchnorm1(res)
        res = self.conv2(res)
        res = torch.renorm(res, p=2, dim=0, maxnorm=1)
        res = self.batchnorm2(res)
        #res = F.dropout(0.25)
        res = self.elu(res)
        res = self.avgPool1(res)
        res = self.dropout(res)
        # Block 2
        res = self.depthwise(res)
        res = self.pointwise(res)
        res = self.batchnorm3(res)
        res = self.elu(res)
        res = self.avgPool2(res)
        res = self.dropout(res)
        # Classifier
        res = torch.flatten(res, start_dim=1)
        res = self.classifier(res)
        res = self.softmax(res)
        return res


## Testing code
Code for testing the EEGNet implementation

In [10]:
network = EEGNet(interval = "action").float().to(device)
accuracy_check(network, train_data, train_labels)

print("####### TRAINING #######")
op = optim.Adam(params = network.parameters(), lr = 0.0001)
lossf = nn.NLLLoss()
train_model(network, train_data = train_data, train_labels = train_labels, val_data = val_data, val_labels = val_labels, epochs = 50, batch_size = 10, loss_func = lossf, optimizer = op)

accuracy_check(network, train_data, train_labels)

ACCURACY: 0.25
####### TRAINING #######
Epoch	 train loss	 validation loss
0 	  1.454275180131961 	 1.3908432722091675
1 	  1.3340616470728166 	 1.3912353515625
2 	  1.309732192601913 	 1.407396912574768
3 	  1.2582181294759114 	 1.4335039854049683
4 	  1.2327967912722857 	 1.431328535079956
5 	  1.1810345771985176 	 1.427665114402771
6 	  1.153683099991236 	 1.4385358095169067
7 	  1.1194413013947315 	 1.4403594732284546
8 	  1.0931360293657353 	 1.4456959962844849
9 	  1.0261296981420271 	 1.4773015975952148
10 	  1.0099297547951722 	 1.4729543924331665
11 	  0.9718165275378104 	 1.502492070198059
12 	  0.932275136311849 	 1.5240556001663208
13 	  0.8948694864908854 	 1.552672028541565
14 	  0.8550456120417669 	 1.5764963626861572
15 	  0.8073525551037911 	 1.5969221591949463
16 	  0.7763902224027194 	 1.6556447744369507
17 	  0.7752760862692808 	 1.6667014360427856
18 	  0.7350969559107071 	 1.656973123550415
19 	  0.6982845893272986 	 1.7046852111816406
20 	  0.6313568017421625 	 1

In [11]:
accuracy_check(network, train_data, train_labels)
accuracy_check(network, val_data, val_labels)
accuracy_check(network, test_data, test_labels)

ACCURACY: 0.9848484848484849
ACCURACY: 0.3148148148148148
ACCURACY: 0.3
